# Import
Importáljuk a könyvtárakat amiket használnunk kell

In [ ]:
import json  # JSON fájlok beolvasása
from datetime import datetime  # Dátumok manipulálására
from hashlib import sha256  # A nevek titkosítására
from pathlib import Path  # Navigálás a fájlrendszerünkben (olyan mint az "os", csak jobb)
from zipfile import ZipFile  # .zip fájlok olvasása

import pandas as pd  # Táblázat kezelés
import matplotlib.pyplot as plt  # A vizualizálásra

# Facebook hiba
Sajnos a Facebook export hibásan kódolja a Unicode karaktereket. Ezt majd meg kell javítanunk. Erra használjuk később ezt a funkciót. Nem kell vele törődni. Ha érdekel a probléma, a doc-string linkjén lehet róla többet olvasni.

In [ ]:
def fix_fb_bytes(data):
    """Source: https://krvtz.net/posts/how-facebook-got-unicode-wrong.html"""
    new_data: bytes = b''
    i: int = 0
    while i < len(data):
        if data[i:].startswith(b'\\u00'):
            u: int = 0
            new_char: bytes = b''
            while data[i+u:].startswith(b'\\u00'):
                hex = int(bytes([data[i+u+4], data[i+u+5]]), 16)
                new_char = b''.join([new_char, bytes([hex])])
                u += 6

            char : str = new_char.decode('utf-8')
            new_chars: bytes = bytes(json.dumps(char).strip('"'), 'ascii')
            new_data += new_chars
            i += u
        else:
            new_data = b''.join([new_data, bytes([data[i]])])
            i += 1

    return new_data

# Beolvasás

In [ ]:
# Megadjuk, hogy hol van a letöltött .zip fájl a rendszerünkön
facebook_data_zipfile = Path('/home/misinagy/Documents/Personal/dt3/facebook-mihalynagy1232-20_09_2025-PvHj805b.zip')

In [ ]:
# Kiszedjük a .zip fájlból azokat a fájl elérési útvonalakat, amelyek message_xyz.json formátumot követik.
message_files = []
with ZipFile(facebook_data_zipfile, 'r') as zip_fh:
    for file in zip_fh.namelist():
        zip_file_filepath = Path(file)
        if zip_file_filepath.name.startswith('message_') and zip_file_filepath.suffix == '.json':
            message_files.append(str(zip_file_filepath))

In [ ]:
# Végigmegyünk a kinyert fájl elérési útvonalakon és kiolvassuk a .zip fájlból a JSON adatokat, letároljuk egy listában az összeset. Ez 2-3 percig eltarthat.
messages_raw = []
with ZipFile(facebook_data_zipfile, 'r') as zip_fh:
    for message_file in message_files:
        data = json.loads(fix_fb_bytes(zip_fh.read(message_file)))  # Itt használjuk a Facebook Unicode hiba kijavítást
        messages_raw.append(data)

# Titkosítás

In [ ]:
# Kinyerjük a saját sha265 kódunkat, hogy be tudjuk azonosítani önmagunkat
mycode = sha256('Mihály Nagy'.encode('utf-8')).hexdigest()
print(f'Mihály Nagy = {mycode}')

In [ ]:
# Végigmegyünk az adatunkban az összes 'messages' objektumon, és kicseréljük a neveket egy sha265 kódra.
# A sha265 egy visszafordíthatatlan hash algoritmus eredménye, így biztos, hogy senki sem fogja tudni kideríteni kivel beszéltünk.
secret_name_code_lookup = {}
other_person = None
table_rows_data = []
for data in messages_raw:
    if len(data['participants']) == 2:  # Itt csak azokat a beszélgetéseket választom ki ahol két ember közt történt a chatelés.
        for participant in data['participants']:
            code = sha256(participant['name'].encode('utf-8')).hexdigest()
            secret_name_code_lookup[code] = participant['name']
            if code != mycode:
                other_person = code
        # Titkosítjuk a message.sender_name változót és kitöröljük a message.content és message.reactions értékeket
        for message in data['messages']:

            # Kitöröljük azokat az értékeket amikre nem lesz szükségünk. (content, reactions, etc.)
            keys_not_needed = [key for key in message.keys() if key not in {'sender_name', 'timestamp_ms'}]
            for key in keys_not_needed:
                message.pop(key, None)

            # Titkosítjuk a küldő nevét
            sender = sha256(message['sender_name'].encode('utf-8')).hexdigest()

            # Ha én voltam a küldő akkor hozzáadjuk a fogadó kódját mint fogadó
            if sender == mycode:
                receiver = other_person
            else:  # ha meg nem, akkor én leszek a fogadó
                receiver = mycode

            table_rows_data.append((sender, receiver, message['timestamp_ms'], other_person))


# Táblázat felépítése

In [ ]:
# Beolvassuk egy pandas DataFrame objektumba az adat listánkat. Megadjuk az oszlopneveket.
message_df = pd.DataFrame(table_rows_data, columns=['sender', 'receiver', 'timestamp_ms', 'interacted_with'])

In [ ]:
# A Dátumokból kinyerjük az év-hónap formátumot és elmentjük a year_month oszlopba
def convert_to_year_month(timestamp):
    return datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m')
message_df['year_month'] = message_df['timestamp_ms'].apply(convert_to_year_month)

In [ ]:
# Megszámoljuk, hogy kivel hányszor interaktáltunk egy év-hónap-ban.
interaction_count_by_person_df = message_df.groupby(['year_month', 'interacted_with']).size().reset_index().rename({0: 'count'}, axis=1)

In [ ]:
# Végül a végső formátumba hozzuk az adatunk. Az év-hónap legyen a bal oldalon az index oszlop, az interacted_with-ből legyen egy külön oszlop minden egyedi kódnak, és a "count" értékei legyenek a cellákban.
final = interaction_count_by_person_df.pivot(index='year_month', columns='interacted_with', values='count')

In [ ]:
# Nézzünk rá hogy néz ki.
final.plot(kind='line', figsize=(12, 4))
plt.legend().remove()

In [ ]:
# Mentsük el a kész adatot.
# Aki akarja, visszaváltoztathatja a neveket az eredetikra
# final = final.rename(secret_name_code_lookup, axis=1)
final.to_csv('fb_interactions_yearmonth.csv')

# Utólagos érdekességek

In [ ]:
# Megnézhetjük kikkel beszéltünk a legtöbbet
interaction_count_by_person_df.groupby('interacted_with')['count'].sum().sort_values(ascending=False)

In [ ]:
# És így visszanézhetjük a kódokhoz tartozó neveket.
secret_name_code_lookup['d5d38a4a804bd900023d541846695afd39cdda279b1857ed6cdc73c40c189cda']